In [36]:
# %load make_walks_10.py
import sys 
sys.path.append("/Users/stiso/Documents/Code/graph_learning/")
import random
import networkx as nx
import numpy as np
#from utilities import graphs
import matplotlib.pyplot as plt
import os
from scipy import stats
import pickle
import functions.make_walks as mw
import csv
import seaborn as sns
from gtts import gTTS
import pyttsx3
import pandas as pd
from gtts import gTTS 
import librosa
import soundfile as sf
import wave

random.seed()

In [2]:
# for 10 nodes
_schapiro10_list = {
    0: [1, 2, 3, 9],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 4],
    3: [0, 1, 2, 4],
    4: [1, 2, 3, 5],
    5: [4, 6, 7, 8],
    6: [5, 7, 8, 9],
    7: [5, 6, 8, 9],
    8: [5, 6, 7, 9],
    9: [6, 7, 8, 0]}

_ring_lattice10_list = {
    0: [8, 9, 1, 2],
    1: [9, 0, 2, 3],
    2: [0, 1, 3, 4],
    3: [1, 2, 4, 5],
    4: [2, 3, 5, 6],
    5: [3, 4, 6, 7],
    6: [4, 5, 7, 8],
    7: [5, 6, 8, 9],
    8: [6, 7, 9, 0],
    9: [7, 8, 0, 1]

}

lattice = nx.from_dict_of_lists(_ring_lattice10_list)
schapiro = nx.from_dict_of_lists(_schapiro10_list)

In [7]:
# define some constants
# For one subject, generate either a random of modular graph
nTrial = 3000 # number of trials in pre-exposire sequence
nTest = 1000 # number of trials in 
distRatio = 0.1 # percentage of test trials that will be distractor words

nSubj = 100
graph_uni = []
csvfiles = ["/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/mod_pre_exp.csv",
           "/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/lat_pre_exp.csv"]
test_csvfiles = ["/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/mod_test.csv",
           "/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/lat_test.csv"]
ext = ['mod_pre_exp', 'lat_pre_exp']
graphs = [schapiro, lattice]
wei_flag = [0,0]

nNode = len(syls)
top_dir = '/Users/stiso/Documents/Code/tasks/graph_learn_aud/'


In [6]:
########################
 #   MAKE THE WALK CSVS
 #########################
 
for j,fi in enumerate(csvfiles):
    # write to csv
    if os.path.isfile(fi):
        # hacky way to get around the fact that I am adding to existing files, and not overwriting them. 
        #Fix this later?
        raise NameError("You have already created walks for this graph. If you would like to make new ones, delete the csv file and try again")

    for i in range(nSubj):
        if wei_flag[j]:
            walks = mw.random_walk_wei(graphs[j],nTrial)
            test_walks = mw.random_walk_wei(graphs[j],nTest)
        else:
            walks = mw.random_walk(graphs[j],nTrial)
            test_walks = mw.random_walk(graphs[j],nTest)
        # save p-value indicating uniformity of walk
        graph_uni.append(stats.kstest(walks, stats.uniform(loc=0.0, scale=9.0).cdf)[0])
    
        # plot and save histogram of walk
        fig = plt.figure()
        sns.distplot(walks, bins=10, hist_kws={"rwidth":0.75})
        plt.xlabel('Node')
        plt.ylabel('Frequency')
        #plt.savefig("".join(['/Users/stiso/Documents/Python/graphLearning/images/', str(i), ext[j], '.png']))
        
        # write pre-exp
        with open(fi, "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(walks)
            
        # write test
        with open(test_csvfiles[j], "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(test_walks)
          

    # Saving the stats about how uniform edge distribution is:
    #with open("".join(['/Users/stiso/Documents/Python/graphLearning/uniformity', ext[j], '.pkl']), 'wb') as f:  # Python 3: open(..., 'wb')
    #    pickle.dump([graph_uni], f)


/Users/stiso/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [5]:
# save each sylable
syls = ['bloy', 'snab', 'jarn', 'fust', 'kank', 'paft', 'tibe', 'wabe', 'yent', 'cose']
distractors = ['zard', 'bulp', 'gount', 'dand', 'dimp', 'gilk', 'jass', 'nuck', 'prill', 'frit']
for p in syls:
    # save as text file
    file = open(f"/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/{p}.txt","w+")
    file.write(p)
    file.close()

    #save auditory
    aud_file = f"/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/{p}.wav"
    # Passing the text and language to the engine,  
    # here we have marked slow=False. Which tells  
    # the module that the converted audio should  
    # have a high speed 
    myobj = gTTS(text=p, lang='en', slow=False) 
    myobj.save(aud_file)
    # save again so that file can be read by wave later
    x,_ = librosa.load(aud_file, sr=16000)
    sf.write(aud_file, x, 16000)
    
for p in distractors:
    # save as text file
    file = open(f"/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/{p}.txt","w+")
    file.write(p)
    file.close()

    #save auditory
    aud_file = f"/Users/stiso/Documents/Code/tasks/graph_learn_aud/stims/{p}.wav"
    # Passing the text and language to the engine,  
    # here we have marked slow=False. Which tells  
    # the module that the converted audio should  
    # have a high speed 
    myobj = gTTS(text=p, lang='en', slow=False) 
    myobj.save(aud_file)
    # save again so that file can be read by wave later
    x,_ = librosa.load(aud_file, sr=16000)
    sf.write(aud_file, x, 16000)

/Users/stiso/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/stiso/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/stiso/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/stiso/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/Users/stiso/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread i

In [53]:
# make the subject specific csvs and data files
maps = []

for j,fi in enumerate(csvfiles):
    data = pd.read_csv(fi)
    test_data = pd.read_csv(test_csvfiles[j])
    for i in range(nSubj-1):
        # get directory of subject csv
        subj_dir = os.path.join(top_dir, f'subjData/subj{i}/')
        if not os.path.isdir(subj_dir):
            os.makedirs(subj_dir)
            
        # get current walk
        curr = data.iloc[i,:]
        curr_test = test_data.iloc[i,:]
        
        # get random mapping
        np.random.shuffle(syls)
        mapping = pd.DataFrame({'sID':i, 'sylNum':range(nNode), 'sylFile':[x + '.txt' for x in syls], 
                                'sylSound':[x + '.wav' for x in syls]})
        map_fi = os.path.join(subj_dir, 'syl_map.csv')
        # and save to csv
        mapping.to_csv(map_fi, index=False)
        
        # save practice walk
        prac_sound = np.array(['stims/' + n for n in mapping['sylSound']])
        prac_test = [n[6:-4] for n in prac_sound]
        # get random distractor word, that starts with a different letter
        prac_distractors = []
        for n,t in enumerate(prac_test):
            curr_dist = random.choice(distractors)
            while curr_dist[0] == t[0]:
                curr_dist = random.choice(distractors)
            prac_distractors.append(curr_dist)
        # pick random trials for distractors to be played
        ind = random.choices(range(nNode), k=3)
        prac_dist_files = ['stims/' + prac_distractors[n] + '.wav' for n in ind]
        prac_sound[ind] = prac_dist_files
        prac_dist_idx = [n in ind for n in range(nNode)]
        # save as df
        prac_df = pd.DataFrame({'sID':i, 'node':range(nNode), 'soundFile':prac_sound, 
                                'testWord':prac_test , 'distWord':prac_distractors, 'dist':prac_dist_idx})
        prac_df.to_csv(subj_dir + 'prac_walk.csv', index=False)
        
        # save full test walk with extra info
        test_files = np.array(['stims/' + mapping.sylSound[n] for n in curr_test])
        test_words = [mapping.sylSound[n][0:-4] for n in curr_test]
        distractor_words = []
        for n,t in enumerate(test_words):
            curr_dist = random.choice(distractors)
            while curr_dist[0] == t[0]:
                curr_dist = random.choice(distractors)
            distractor_words.append(curr_dist)
        # pick indices for distractor trials
        ind = random.choices(range(nTest), k=int(nTest*distRatio))
        dist_files = ['stims/' + distractor_words[n] + '.wav' for n in ind]
        test_files[ind] = dist_files
        dist_idx = [n in ind for n in range(nTest)]
        test_df = pd.DataFrame({'sID':i, 'node':curr_test, 'soundFile':test_files, 
                                'testWord':test_words , 'distWord':distractor_words, 'dist':dist_idx})
        test_df.to_csv(subj_dir + 'test_walk.csv', index=False)
        
        # save pre-exp sequence as text file
        curr_text = [mapping.sylSound[n][0:-4] for n in curr]
        walk_files = [top_dir + 'stims/' + mapping.sylSound[n] for n in curr]
        file = open("".join([subj_dir, f"/stream.txt"]),"w+")
        file.write(", ".join(curr_text))
        file.close()
        
        # save auditory pre-exp seq
        aud_file = "".join([subj_dir, f"stream.wav"])
        with wave.open(aud_file, 'wb') as wav_out:
            for wav_path in walk_files:
                with wave.open(wav_path, 'rb') as wav_in:
                    if not wav_out.getnframes():
                        wav_out.setparams(wav_in.getparams())
                    wav_out.writeframes(wav_in.readframes(wav_in.getnframes())) 


In [62]:
CHANNELS = 1
swidth = 2
Change_RATE = 1.1

spf = wave.open('/Users/stiso/Documents/Code/tasks/graph_learn_aud/subjData/subj3/stream.wav', 'rb')
RATE=spf.getframerate()
signal = spf.readframes(-1)

wf = wave.open('/Users/stiso/Documents/Code/tasks/graph_learn_aud/subjData/subj3/stream.wav', 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(swidth)
wf.setframerate(RATE*Change_RATE)
wf.writeframes(signal)
wf.close()

In [ ]:
espeak_path = "C:/Program Files/eSpeak/command_line/espeak.exe"
file_name = "C:/temp/test"
subprocess.call([espeak_path,"-w"+file_name+".wav", text])